In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
%cd drive/MyDrive/tabu

[Errno 2] No such file or directory: 'drive/MyDrive/tabu'
/content/drive/MyDrive/tabu


In [71]:
import numpy as np
import copy
import random 
import os
import time
import json
import glob

# Input

In [72]:
def loadData(path):
    f = open(path, "r")
    data = json.load(f)

    N, m, M, d, s, e = data["N"], data["m"], data["M"], data["d"], data["s"], data["e"]
    date = np.max(e)+1
    # print(d)
    # print(s)
    # print(e)
    # print(date)
    return  N, m, M, d, s, e, date

In [32]:
# N, m, M, d, s, e, date = loadData('sample_50_50_100.json')
# seed = 5

# Cost function


In [73]:
def getCost(N, m, M, d, date, sol):
    amount = np.zeros(date)
    penalty = 0
    for i in range(N):
        amount[sol[i]] += int(d[i])
    for i in amount:
        if (i > M):
            penalty += i-M
        elif (i > 0 and i < m):
            penalty += m-i
    amount_temp = np.delete(amount, amount == 0)
    cost = np.max(amount_temp)-np.min(amount_temp)
    return int(cost+1000*penalty)

In [74]:
def Cost(N, m, M, d, date, sol):
    amount = np.zeros(date)
    for i in range(N):
        amount[sol[i]] += int(d[i])
    amount_temp = np.delete(amount, amount == 0)
    cost = np.max(amount_temp)-np.min(amount_temp)
    return int(cost)

# Initialize solution

In [75]:
def initialSolution(N, m, M, d, s, e, date, SEED):
    random.seed(SEED)
    sol = [0]*N
    amount = np.zeros(date)
    global flag
    flag = [0]*date

    # cánh đồng nào thu nhiều thì ưu tiên chọn trước
    order = np.argsort(-np.array(d))
    for i in order:
        temp1 = [j for j in range(s[i], e[i]+1) if flag[j] <= 1]
        temp2 = [j for j in range(s[i], e[i]+1) if (flag[j] == 2 and amount[j] + d[i] <= M)]
        if (len(temp1) != 0):
            day = random.choice(temp1)
        elif (len(temp2) != 0):
            day = random.choice(temp2)
        else:
            day = random.randint(s[i], e[i])

        sol[i] = day
        amount[day] += d[i]
        if (amount[day] > M):
            flag[day] = 3  # đã đến giới hạn
        elif (amount[day] >= m):
            flag[day] = 2  # vừa đủ
        elif (amount[day] > 0):
            flag[day] = 1  # chưa đủ         
    return sol

In [ ]:
# initialSolution(N, m, M, d, s, e, date, SEED=seed)

# Tabu Search

## Search for next solution

In [77]:
def search_next_solution(N, m, M, d, s, e, date, sol, SEED):    
    random.seed(SEED)   
    select_field = -1
    select_day = -1
    new_search_objective = 1e9
    global flag
    flag = [0]*date

    amount = np.zeros(date)
    penalty =0
    for i in range(N):
        amount[sol[i]]+=d[i]
    for i in range(date):   
        if (amount[i] > M):
            penalty += (amount[i] - M)
        elif (amount[i] > 0 and amount[i] < m):
            penalty += (m - amount[i])
        # penalty+=max(amount[i]-M,0)+max(0,min(amount[i],m-amount[i]))

        if (amount[i]>M):
            flag[i]=3 #đã đến giới hạn
        elif (amount[i]>=m):
            flag[i]=2
        elif (amount[i]>0):
            flag[i]=1
    # cost = np.max(amount)-np.min(amount)+1000*penalty
    
    randomOrder = np.arange(N)
    random.shuffle( randomOrder)
    for i in  randomOrder:
        if tabuList[i] > 0:
            continue             
        for j in range(s[i],e[i]+1):
            if (j == sol[i] or flag[j]==3):
                continue    

            # get old penalty
            oldPenalty = 0
            ## penalty for day sol[i]
            if (amount[sol[i]] > M):
                oldPenalty += (amount[sol[i]] - M)
            elif (amount[sol[i]] > 0 and amount[sol[i]] < m):
                oldPenalty += (m - amount[sol[i]])
            ## penalty for day j
            if (amount[j] > M):
                oldPenalty += (amount[j] - M)
            elif (amount[j] > 0 and amount[j] < m):
                oldPenalty += (m - amount[j])
            # change harvest day of i-th field from sol[i]-th day to j-th day
            amount[sol[i]]-=d[i]
            amount[j]+=d[i]

            # get new penalty
            newPenalty = 0
            ## penalty for day sol[i]
            if (amount[sol[i]] > M):
                newPenalty += (amount[sol[i]] - M)
            elif (amount[sol[i]] > 0 and amount[sol[i]] < m):
                newPenalty += (m - amount[sol[i]])
            ## penalty for day j
            if (amount[j] > M):
                newPenalty += (amount[j] - M)
            elif (amount[j] > 0 and amount[j] < m):
                newPenalty += (m - amount[j])

            # compute new objective
            amount_temp = np.delete(amount, amount == 0)
            newSolObjective = np.max(amount_temp)-np.min(amount_temp) + 1000*(penalty-oldPenalty+newPenalty)
            #restore
            amount[sol[i]]+=d[i]
            amount[j]-=d[i]

            if newSolObjective < new_search_objective:
                new_search_objective = newSolObjective
                select_field = i
                select_day = j

    return select_field, select_day, new_search_objective

## TabuSearch Algorithm

In [78]:
def TabuSearch(N, m, M, d, s, e, date, SEED):   
    solve = initialSolution(N, m, M, d, s, e, date, SEED)
    bestSolve = np.array(solve)
    
    global tabuList
    itmax = 500
    it=0
    tabuList = np.zeros(N)
    tbl = 7
    tbl_min = 5
    tbl_max = 9
    stable = 0
    stable_limit = 10

    while (it < itmax):
        it+=1

        #Move
        # oldSolve = np.array(solve)
        old_search_objective = getCost(N, m, M, d, date, solve)
        field, day, new_search_objective = search_next_solution(N, m, M, d, s, e, date, solve, SEED+it)
        solve[field] = day
        # search_objective = new_search_objective

        #Update tabu
        for i in range(N):
            if tabuList[i] > 0:
                tabuList[i] -= 1
        tabuList[field] = tbl

        if new_search_objective < old_search_objective:
            if tbl > tbl_min:
                tbl -= 1
        else:
            if tbl < tbl_max:
                tbl += 1

        #Update last improved solution
        if new_search_objective < old_search_objective:
            # print("Improved")
            stable = 0
        else:
            stable += 1

        #Cập nhật lời giải kỉ lục
        if new_search_objective < getCost(N, m, M, d, date, bestSolve):
            bestSolve = np.array(solve)
        elif stable == stable_limit:   
            stable = 0
            # Chuyển sang không gian khác
            solve = initialSolution(N, m, M, d, s, e, date, SEED+it)
            #Update tabu
            tabuList = np.zeros(N)
    return bestSolve

In [ ]:
# TabuSearch('123.txt', '50_50_100',SEED = 10)

#Log output

In [79]:
def logOutput(outputPath, Name, time, N, d, date, SEED, sol):
    f = open(outputPath,"w")
    f.write("Name: "+Name+"\n")
    f.write("Time: {} \n".format(time))
    f.write("Result: {}\n".format(Cost(N, m, M, d, date, sol)))
    f.write("Solution: ")
    for i in sol:
        f.write(" {}".format(i))
    f.write("\n")
    amount = np.zeros(date)
    day = [[] for i in range(date)]
    for i in range(N):
        amount[sol[i]]+=d[i]
        day[sol[i]].append(i)

    for i in range(date):
        f.write("{} : {} ({})\n".format(i,day[i],amount[i]))
        # print("----sum: {}".format(amount[i]))

In [80]:
def logOutputWithPenalty(outputPath, Name, time, N, d, date, SEED, sol):
    f = open(outputPath,"w")
    f.write("Name: "+Name+"\n")
    f.write("Time: {} \n".format(time))
    f.write("Result: {}\n".format(getCost(N, m, M, d, date, sol)))
    

# Test 

In [81]:
import os
import time
for path in glob.glob(os.path.join('.', 'input', '**')):
    data_type = path.split('/')[-1]
    input_path = os.path.join('.', 'input', data_type)
    output_path = os.path.join('.', 'TabuSearch', data_type)
    # penalty_path = os.path.join('.', 'penalty', data_type)
    if not os.path.exists(output_path):
        os.makedirs(output_path, exist_ok=True)
    # if not os.path.exists(penalty_path):
    #     os.makedirs(penalty_path, exist_ok=True)
    for path in glob.glob(os.path.join(input_path, '**.json')):
        print('-------START-------')
        input_file = path.split('/')[-1]
        name = input_file[7:-5]
        print(path)
        print(name)
        N, m, M, d, s, e, date = loadData(os.path.join(input_path, input_file))
        seed = 5
        start = time.time()
        sol = TabuSearch(N, m, M, d, s, e, date, SEED=seed)
        end = time.time()
        logOutput(outputPath=os.path.join(output_path, 'TS({}).txt'.format(name)), Name=name, time=end-start, N=N, d=d, date=date, SEED=seed, sol=sol)
        # logOutputWithPenalty(outputPath=os.path.join(penalty_path, 'TS({}).txt'.format(name)), Name=name, time=end-start, N=N, d=d, date=date, SEED=seed, sol=sol)
        print('--------END--------')
    # break


-------START-------
./input/Type1Large/sample_100_40_75.json
100_40_75
--------END--------
-------START-------
./input/Type1Large/sample_100_50_100.json
100_50_100
--------END--------
-------START-------
./input/Type1Large/sample_100_60_125.json
100_60_125
--------END--------
-------START-------
./input/Type1Large/sample_100_30_50.json
100_30_50
--------END--------
-------START-------
./input/Type1Large/sample_100_20_30.json
100_20_30
--------END--------
-------START-------
./input/Type1Large/sample_1000_60_125.json
1000_60_125
--------END--------
-------START-------
./input/Type1Large/sample_1000_40_75.json
1000_40_75
--------END--------
-------START-------
./input/Type1Large/sample_1000_50_100.json
1000_50_100
--------END--------
-------START-------
./input/Type1Large/sample_1000_30_50.json
1000_30_50
--------END--------
-------START-------
./input/Type1Large/sample_1000_20_30.json
1000_20_30
--------END--------
-------START-------
./input/Type1Large/sample_750_50_100.json
750_50_100

In [36]:
for path in glob.glob(os.path.join('.', 'input', '**')):
    print(path)

./input/Type1Large
./input/HomologousField
./input/Type1Small
./input/Type5Large
./input/Type4Small
./input/Type4Large
./input/Type2Small
./input/Type2Large
./input/Type5Small
./input/Type3Large
./input/Type3Small
